In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_csv("files/handm.csv")

df.head()

,Unnamed: 0,productId,productName,brandName,url,price,stockState,comingSoon,colorName,isOnline,colors,colorShades,newArrival,mainCatCode,details,materials
0,0,834333001,Derby Shoes,H&M,https://www2.hm.com/en_us/productpage.08343330...,49.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_dressed,Derby shoes with open lacing at front. Canvas ...,COMPOSITION\nSole:Thermoplastic rubber 100%\nU...
1,1,1258600003,Pile-Lined Slippers,H&M,https://www2.hm.com/en_us/productpage.12586000...,24.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_slippers,Slippers in napped fabric. Decorative seam at ...,COMPOSITION\nLining:Polyester 100%\nUpper:Poly...
2,2,1036812001,Derby Shoes,H&M,https://www2.hm.com/en_us/productpage.10368120...,56.99,Available,False,Black,True,272628,Charcoal,False,men_shoes_dressed,Derby shoes in soft faux suede with open lacin...,COMPOSITION\nLining and insole:Cotton 100%\nUp...
3,3,1112924002,Fleece-lined Slippers,H&M,https://www2.hm.com/en_us/productpage.11129240...,29.99,Available,False,Dark blue,True,393B47,Bright blue|Cobalt blue|Dark blue|Indigo|Navy ...,False,men_shoes_slippers,Slippers in woven fabric with a moccasin seam ...,COMPOSITION\nLining:Polyester 100%\nUpper:Poly...
4,4,1119476020,Sneakers,H&M,https://www2.hm.com/en_us/productpage.11194760...,29.99,Available,False,White,True,FFFFFF,NaN,False,men_shoes_sneakers,"Sneakers with a padded upper edge and tongue, ...",COMPOSITION\nLining:Polyester 100%\nUpper:Poly...


In [3]:
df = df.drop(columns=['Unnamed: 0'])

In [4]:
df['materials'].sample(5)

2327    COMPOSITION\nShell:Linen 100% , Material:\nLin...
5421    COMPOSITION\nShell:Polyester 80%, Spandex 20%\...
6301    COMPOSITION\nPolyester 75%, Spandex 25% , ADDI...
4218    COMPOSITION\nShell:Cotton 100%\nPocket lining:...
1899    COMPOSITION\nLyocell 100% , ADDITIONAL MATERIA...
Name: materials, dtype: object

In [5]:
#Crear bucle para que cuando 'organic', 'recycled' lo marque

df['recycled'] = df['materials'].apply(
    lambda x: 'Yes' if 'recycled' in str(x).lower() else 'No')

In [6]:
df['organic'] = df['materials'].apply(
    lambda x: 'Yes' if 'organic' in str(x).lower() else 'No')

In [7]:
df.shape

(9677, 17)

In [8]:
df['recycled'].value_counts()

recycled
Yes    5869
No     3808
Name: count, dtype: int64

In [9]:
df['organic'].value_counts()

organic
No     9522
Yes     155
Name: count, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9677 entries, 0 to 9676
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   productId    9677 non-null   int64  
 1   productName  9677 non-null   object 
 2   brandName    9677 non-null   object 
 3   url          9677 non-null   object 
 4   price        9677 non-null   float64
 5   stockState   9677 non-null   object 
 6   comingSoon   9677 non-null   bool   
 7   colorName    9677 non-null   object 
 8   isOnline     9677 non-null   bool   
 9   colors       9677 non-null   object 
 10  colorShades  6511 non-null   object 
 11  newArrival   9677 non-null   bool   
 12  mainCatCode  9677 non-null   object 
 13  details      9676 non-null   object 
 14  materials    9518 non-null   object 
 15  recycled     9677 non-null   object 
 16  organic      9677 non-null   object 
dtypes: bool(3), float64(1), int64(1), object(12)
memory usage: 1.1+ MB


In [11]:
df['colorName']

0                  Black
1                  Black
2                  Black
3              Dark blue
4                  White
              ...       
9672               Beige
9673    Light denim blue
9674         Powder pink
9675         Beige/black
9676               Brown
Name: colorName, Length: 9677, dtype: object

In [12]:
for columna in df:
    print(f'La columna {columna} tiene estos valores únicos:')
    print(df[columna].unique())
    print('----------------------------')

La columna productId tiene estos valores únicos:
[ 834333001 1258600003 1036812001 ... 1232588003 1208310002 1227825002]
----------------------------
La columna productName tiene estos valores únicos:
['Derby Shoes' 'Pile-Lined Slippers' 'Fleece-lined Slippers' ...
 'Underwire Lace Bra' 'Slim Straight Ultra High Jeans'
 '2-pack Padded Soft-cup Cotton Bras']
----------------------------
La columna brandName tiene estos valores únicos:
['H&M']
----------------------------
La columna url tiene estos valores únicos:
['https://www2.hm.com/en_us/productpage.0834333001.html'
 'https://www2.hm.com/en_us/productpage.1258600003.html'
 'https://www2.hm.com/en_us/productpage.1036812001.html' ...
 'https://www2.hm.com/en_us/productpage.1232588003.html'
 'https://www2.hm.com/en_us/productpage.1208310002.html'
 'https://www2.hm.com/en_us/productpage.1227825002.html']
----------------------------
La columna price tiene estos valores únicos:
[ 49.99  24.99  56.99  29.99  59.99  34.99  54.99  74.99 119.

In [13]:
#Quitamos isOnline porque solo tiene un valor único
df = df.drop(columns='isOnline')

In [14]:
df.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic'],
      dtype='object')

In [15]:
df["mainCatCode"].value_counts()

mainCatCode
ladies_cardigansjumpers_jumpers         361
men_shirts_casual                       250
ladies_dresses_shortdresses             170
ladies_tops_longsleeve                  164
ladies_tops_shortsleeve                 162
                                       ... 
sportswear_men_activity_hiking            1
beauty_brushestools_skincaretools         1
men_hoodiessweatshirts                    1
ladies_shoes                              1
sportswear_women_clothing_tops_tanks      1
Name: count, Length: 355, dtype: int64

In [16]:

# Separar la columna 'categoria' por el carácter "_" en columnas nuevas
df_split = df["mainCatCode"].str.split("_", expand=True)

# Renombrar las nuevas columnas con nombres genéricos (puedes personalizar)
df_split.columns = [f"nivel_{i+1}" for i in range(df_split.shape[1])]

# Unir al DataFrame original si lo deseas
df= pd.concat([df, df_split], axis=1)


In [17]:
df.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic', 'nivel_1',
       'nivel_2', 'nivel_3', 'nivel_4', 'nivel_5'],
      dtype='object')

In [18]:
df[['nivel_1', 'nivel_2', 'nivel_3', 'nivel_4', 'nivel_5']]

,nivel_1,nivel_2,nivel_3,nivel_4,nivel_5
0,men,shoes,dressed,None,None
1,men,shoes,slippers,None,None
2,men,shoes,dressed,None,None
3,men,shoes,slippers,None,None
4,men,shoes,sneakers,None,None
...,...,...,...,...,...
9672,ladies,jeans,straight,None,None
9673,ladies,jeans,bootcut,None,None
9674,ladies,lingerie,bras,padded,None
9675,ladies,lingerie,bras,multipack,None


In [19]:
df['nivel_1'].value_counts()


nivel_1
ladies        6395
men           2236
sportswear     566
beauty         226
Ladies         181
MEN             55
home            13
kids             5
Name: count, dtype: int64

In [20]:

cols_to_drop = df.columns[-3:] 
df_sin_ultimos_niveles = df.drop(columns=cols_to_drop)

print(df_sin_ultimos_niveles)

       productId                         productName brandName  \
0      834333001                         Derby Shoes       H&M   
1     1258600003                 Pile-Lined Slippers       H&M   
2     1036812001                         Derby Shoes       H&M   
3     1112924002               Fleece-lined Slippers       H&M   
4     1119476020                            Sneakers       H&M   
...          ...                                 ...       ...   
9672  1199187014            Slim Straight High Jeans       H&M   
9673  1198343002                  Bootcut High Jeans       H&M   
9674  1232588003                    Lace Push-up Bra       H&M   
9675  1208310002  2-pack Padded Soft-cup Cotton Bras       H&M   
9676  1227825002        Padded Balconette Bikini Top       H&M   

                                                    url  price stockState  \
0     https://www2.hm.com/en_us/productpage.08343330...  49.99  Available   
1     https://www2.hm.com/en_us/productpage.12586000.

In [21]:
df_sin_ultimos_niveles.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic', 'nivel_1',
       'nivel_2'],
      dtype='object')

In [22]:
df = df_sin_ultimos_niveles

In [23]:
df.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic', 'nivel_1',
       'nivel_2'],
      dtype='object')

In [24]:
lista = ['nivel_1', 'nivel_2']
for col in lista:
    df[col] = df[col].str.capitalize()


In [25]:
df[lista].value_counts()

nivel_1     nivel_2            
Ladies      Dresses                743
            Tops                   687
            Cardigansjumpers       510
            Accessories            485
            Basics                 424
            Trousers               419
Men         Tshirtstanks           396
Ladies      Maternity              384
            Lingerie               383
Sportswear  Women                  333
Ladies      Swimwear               297
Men         Shirts                 289
Ladies      Jeans                  287
Men         Jacketscoats           282
Ladies      Jacketscoats           274
            Shirtsblouses          270
            Shoes                  268
Sportswear  Men                    233
Men         Hoodiessweatshirts     233
Ladies      Nightwear              215
Men         Trousers               212
Ladies      Skirts                 210
            Premium                190
Men         Cardigansjumpers       175
            Socks               

In [26]:
df.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic', 'nivel_1',
       'nivel_2'],
      dtype='object')

In [29]:
columnas = ['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic',
       'costumerSegment', 'productType']

In [30]:
df.columns = columnas

In [31]:
df.columns

Index(['productId', 'productName', 'brandName', 'url', 'price', 'stockState',
       'comingSoon', 'colorName', 'colors', 'colorShades', 'newArrival',
       'mainCatCode', 'details', 'materials', 'recycled', 'organic',
       'costumerSegment', 'productType'],
      dtype='object')

In [32]:
df.to_csv('files/products_hm_clean.csv', index=False)

In [31]:
df_brands = pd.read_csv("files/sustainable_fashion_trends_2024.csv")

df_brands.head()

,Brand_ID,Brand_Name,Country,Year,Sustainability_Rating,Material_Type,Eco_Friendly_Manufacturing,Carbon_Footprint_MT,Water_Usage_Liters,Waste_Production_KG,Recycling_Programs,Product_Lines,Average_Price_USD,Market_Trend,Certifications
0,BRAND-0001,Brand_1,Australia,2018,D,Tencel,No,1.75,4511152.79,97844.11,No,2,38.33,Growing,GOTS
1,BRAND-0002,Brand_2,Japan,2015,D,Vegan Leather,Yes,124.39,1951566.31,37267.75,No,15,250.07,Growing,GOTS
2,BRAND-0003,Brand_3,USA,2024,A,Vegan Leather,No,336.66,467454.52,38385.92,No,2,146.16,Growing,B Corp
3,BRAND-0004,Brand_4,Italy,2023,D,Bamboo Fabric,No,152.04,899576.90,32665.45,No,13,165.52,Stable,OEKO-TEX
4,BRAND-0005,Brand_5,USA,2016,D,Bamboo Fabric,Yes,415.63,1809219.90,37295.47,Yes,19,211.63,Stable,Fair Trade


In [32]:
df_brands['Certifications'].unique()

array(['GOTS', 'B Corp', 'OEKO-TEX', 'Fair Trade', nan], dtype=object)

In [33]:
for columna in df_brands:
    print(f'La columna {columna} tiene estos valores únicos:')
    print(df_brands[columna].unique())
    print('----------------------------')

La columna Brand_ID tiene estos valores únicos:
['BRAND-0001' 'BRAND-0002' 'BRAND-0003' ... 'BRAND-4998' 'BRAND-4999'
 'BRAND-5000']
----------------------------
La columna Brand_Name tiene estos valores únicos:
['Brand_1' 'Brand_2' 'Brand_3' ... 'Brand_4998' 'Brand_4999' 'Brand_5000']
----------------------------
La columna Country tiene estos valores únicos:
['Australia' 'Japan' 'USA' 'Italy' 'Brazil' 'France' 'India' 'UK'
 'Germany' 'China']
----------------------------
La columna Year tiene estos valores únicos:
[2018 2015 2024 2023 2016 2017 2022 2011 2012 2013 2020 2021 2014 2010
 2019]
----------------------------
La columna Sustainability_Rating tiene estos valores únicos:
['D' 'A' 'B' 'C']
----------------------------
La columna Material_Type tiene estos valores únicos:
['Tencel' 'Vegan Leather' 'Bamboo Fabric' 'Recycled Polyester' 'Hemp'
 'Organic Cotton']
----------------------------
La columna Eco_Friendly_Manufacturing tiene estos valores únicos:
['No' 'Yes']
-------------

In [34]:
df_brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Brand_ID                    5000 non-null   object 
 1   Brand_Name                  5000 non-null   object 
 2   Country                     5000 non-null   object 
 3   Year                        5000 non-null   int64  
 4   Sustainability_Rating       5000 non-null   object 
 5   Material_Type               5000 non-null   object 
 6   Eco_Friendly_Manufacturing  5000 non-null   object 
 7   Carbon_Footprint_MT         5000 non-null   float64
 8   Water_Usage_Liters          5000 non-null   float64
 9   Waste_Production_KG         5000 non-null   float64
 10  Recycling_Programs          5000 non-null   object 
 11  Product_Lines               5000 non-null   int64  
 12  Average_Price_USD           5000 non-null   float64
 13  Market_Trend                5000 

In [35]:
df_brands.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,5000.0,2.016956e+03,4.347466e+00,2010.00,2.013000e+03,2017.000,2021.000,2024.00
Carbon_Footprint_MT,5000.0,2.503176e+02,1.428023e+02,1.04,1.266150e+02,250.650,372.255,499.93
Water_Usage_Liters,5000.0,2.517862e+06,1.429043e+06,50106.18,1.293087e+06,2499095.995,3763859.550,4999596.61
Waste_Production_KG,5000.0,5.010730e+04,2.874638e+04,1026.38,2.534080e+04,50466.340,74984.650,99947.85
Product_Lines,5000.0,1.043120e+01,5.706573e+00,1.00,5.000000e+00,10.000,15.000,20.00
Average_Price_USD,5000.0,2.593542e+02,1.377310e+02,20.02,1.428650e+02,258.615,378.595,499.94


In [36]:
df.isna().sum()

Unnamed: 0            0
productId             0
productName           0
brandName             0
url                   0
price                 0
stockState            0
comingSoon            0
colorName             0
isOnline           3166
colors                0
colorShades           0
newArrival            1
mainCatCode         159
details               0
materials             0
costumerSegment       0
productType           0
dtype: int64

In [37]:
df_brands.isna().sum()

Brand_ID                         0
Brand_Name                       0
Country                          0
Year                             0
Sustainability_Rating            0
Material_Type                    0
Eco_Friendly_Manufacturing       0
Carbon_Footprint_MT              0
Water_Usage_Liters               0
Waste_Production_KG              0
Recycling_Programs               0
Product_Lines                    0
Average_Price_USD                0
Market_Trend                     0
Certifications                1004
dtype: int64

In [39]:
df_shops = pd.read_csv("files/HM_all_stores.csv")

df_shops.head()

,storeCode,storeClass,name,phone,city,country,countryCode,longitude,latitude,timeZoneIndex,Mon_open_hours,Tue_open_hours,Wed_open_hours,Thu_open_hours,Fri_open_hours,Sat_open_hours,Sun_open_hours,streetName1,streetName2,state,address_string
0,AE0122,Red,Mirdiff city center,+971-42316646,Dubai,United Arab Emirates,AE,55.424840,25.226280,165.0,10:00-22:00,10:00-22:00,10:00-22:00,10:00-23:55,10:00-23:55,10:00-23:55,10:00-22:00,Mirdiff city center,Sheikh Mohammad Bin Zayed Road,Dubai,Mirdiff city center;Sheikh Mohammad Bin Zayed ...
1,AE0149,Flagship,Dubai Mall,+971-44190346,Dubai,United Arab Emirates,AE,55.278446,25.197506,165.0,10:00-23:00,10:00-23:00,10:00-23:00,10:00-23:55,10:00-23:55,10:00-23:00,10:00-23:00,Dubai Mall,Sheikh Zayed Road,Dubai,Dubai Mall;Sheikh Zayed Road;Dubai;Dubai;Dubai
2,AE0209,Blue,Al Markaziyah,+971-26120870,Abu Dhabi,United Arab Emirates,AE,54.357462,24.487245,165.0,10:00-22:00,10:00-22:00,10:00-22:00,10:00-23:00,10:00-23:00,10:00-22:00,10:00-22:00,Al Markaziyah,World Trade Center Mall,Abu Dhabi,Al Markaziyah;World Trade Center Mall;123;Abu ...
3,AE0223,Blue,Abu Dhabi-Dubai Rd - Abu Dhabi,+971-26120851,Abu Dhabi,United Arab Emirates,AE,54.671499,24.523875,165.0,10:00-22:00,10:00-22:00,10:00-22:00,10:00-23:55,10:00-23:55,10:00-20:00,10:00-22:00,Abu Dhabi-Dubai Rd - Abu Dhabi,Al Bahia - Al Shahama,Abu Dhabi,Abu Dhabi-Dubai Rd - Abu Dhabi;Al Bahia - Al S...
4,AE0273,Flagship,Yas Mall,+971-24926530,Abu Dhabi,United Arab Emirates,AE,54.609720,24.488684,165.0,10:00-22:00,10:00-22:00,10:00-22:00,10:00-23:55,10:00-23:55,10:00-22:00,10:00-22:00,Yas Mall,NaN,Abu Dhabi,Yas Mall;;x;Abu Dhabi


In [40]:
df_shops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4292 entries, 0 to 4291
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   storeCode       4292 non-null   object 
 1   storeClass      4289 non-null   object 
 2   name            4290 non-null   object 
 3   phone           4140 non-null   object 
 4   city            4292 non-null   object 
 5   country         4292 non-null   object 
 6   countryCode     4292 non-null   object 
 7   longitude       4292 non-null   float64
 8   latitude        4292 non-null   float64
 9   timeZoneIndex   4289 non-null   float64
 10  Mon_open_hours  4258 non-null   object 
 11  Tue_open_hours  4258 non-null   object 
 12  Wed_open_hours  4258 non-null   object 
 13  Thu_open_hours  4258 non-null   object 
 14  Fri_open_hours  4258 non-null   object 
 15  Sat_open_hours  4258 non-null   object 
 16  Sun_open_hours  3148 non-null   object 
 17  streetName1     4292 non-null   o

In [41]:
for columna in df_shops:
    print(f'La columna {columna} tiene estos valores únicos:')
    print(df_shops[columna].unique())
    print('----------------------------')

La columna storeCode tiene estos valores únicos:
['AE0122' 'AE0149' 'AE0209' ... 'ZA0030' 'ZA0032' 'ZA0040']
----------------------------
La columna storeClass tiene estos valores únicos:
['Red' 'Flagship' 'Blue' 'F' nan]
----------------------------
La columna name tiene estos valores únicos:
['Mirdiff city center' 'Dubai Mall' 'Al Markaziyah' ...
 'Cresta Shopping Centre' 'Eikestad Mall' 'Westgate Shopping Centre']
----------------------------
La columna phone tiene estos valores únicos:
['+971-42316646' '+971-44190346' '+971-26120870' ... '+84-2862856089'
 '+84-2836220697' '+27-860690707']
----------------------------
La columna city tiene estos valores únicos:
['Dubai' 'Abu Dhabi' 'Fujairah' ... 'Richards Bay' 'East London'
 'Bloemfontein']
----------------------------
La columna country tiene estos valores únicos:
['United Arab Emirates' 'Austria' 'Australia' 'Bosnia and Herzegovina'
 'Belgium' 'Bulgaria' 'Bahrain' 'Belarus' 'Canada' 'Switzerland' 'Chile'
 'Mainland China' 'Colomb

In [42]:
df_shops[df_shops['Sun_open_hours'].isna() == True].head()

,storeCode,storeClass,name,phone,city,country,countryCode,longitude,latitude,timeZoneIndex,Mon_open_hours,Tue_open_hours,Wed_open_hours,Thu_open_hours,Fri_open_hours,Sat_open_hours,Sun_open_hours,streetName1,streetName2,state,address_string
32,AE1435,Red,The Dubai Mall,NaN,Dubai,United Arab Emirates,AE,55.279791,25.199586,165.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,The Dubai Mall,Financial Center Road,Dubai,The Dubai Mall;Financial Center Road;X;Dubai
33,AT0003,Red,Forum 1,+43-8006655900,Salzburg,Austria,AT,13.044186,47.814383,100.0,09:00-19:00,09:00-19:00,09:00-19:00,09:00-19:00,09:00-19:00,09:00-18:00,NaN,Südtiroler Platz 11,NaN,Salzburg,Südtiroler Platz 11;;5020;Salzburg;Salzburg
34,AT0004,Blue,Stadtgalerien Schwaz,+43-8006655900,Schwaz,Austria,AT,11.706930,47.347280,100.0,09:00-19:00,09:00-19:00,09:00-19:00,09:00-19:00,09:00-19:00,09:00-18:00,NaN,Andreas Hofer-Strasse 10,NaN,Tirol,Andreas Hofer-Strasse 10;;6130;Schwaz;Tirol
35,AT0005,Blue,Shopping Nord,+43-8006655900,Graz,Austria,AT,15.398302,47.104565,100.0,09:00-19:00,09:00-19:00,09:00-19:00,09:00-19:00,09:00-19:00,09:00-18:00,NaN,Wiener Straße 351,NaN,Steiermark,Wiener Straße 351;;8051;Graz;Steiermark
36,AT0006,Red,G3 Shopping Resort Gerasdorf,+43-8006655900,Gerasdorf,Austria,AT,16.462314,48.342263,100.0,09:30-19:00,09:30-19:00,09:30-19:00,09:30-20:00,09:30-20:00,09:00-18:00,NaN,G3 Platz 1,NaN,Niederösterreich,G3 Platz 1;;2201;Gerasdorf bei Wien;Niederöste...


In [43]:
#Bucle para sacar si en todos los días hay NaN porque no tiene los datos de horario o si es un día en concreto que cierran.